In [1]:
import openai
from langchain.llms.base import LLM
from typing import Optional, List, Union
from dotenv import load_dotenv
import os
from loguru import logger

load_dotenv('../.env')

logger.add("../logs/doc_rag.log", rotation="5 MB", level="DEBUG",
           format="{time:YYYY-MM-DD at HH:mm:ss} | {level} | {message}")


MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY") # в .env вставить ключ
MISTRAL_API_URL = "https://api.mistral.ai/v1/"

class MistralLLM(LLM):
    api_key: str
    model_name: str
    api_url: str

    @property
    def _llm_type(self) -> str:
        return "mistral"

    def _call(self, system_prompt: str, user_prompt: str,
              stop: Optional[List[str]] = None, max_tokens: int = 150, **kwargs) -> str:
        client = openai.Client(api_key=self.api_key, base_url=self.api_url)
        payload = {
            "model": self.model_name,
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            "max_tokens": max_tokens,
            **kwargs
        }
        logger.debug("Request Payload: {}", payload)
        try:
            response = client.chat.completions.create(**payload)
            logger.debug("Response: {}", response)
            return response.choices[0].message.content
        except Exception as e:
            logger.error("Error: {}", e)
            raise

    def generate(self, system_prompt: str, user_prompt: str, **kwargs) -> str:
        return self._call(system_prompt, user_prompt, **kwargs)

In [3]:
from langchain import PromptTemplate

mistral_llm = MistralLLM(api_key=MISTRAL_API_KEY,
                         model_name="mistral-large-latest",
                         api_url=MISTRAL_API_URL)

system_template = """You are an assistant that helps with user requests.
Be as helpful as possible and return the answer in Russian language."""

user_template = "{input_text}"


def generate_response(user_input: str, max_tokens: int = 512) -> str:
    system_prompt_template = PromptTemplate(input_variables=[], template=system_template)
    user_prompt_template = PromptTemplate(input_variables=["input_text"], template=user_template)

    formatted_system_prompt = system_prompt_template.format()
    formatted_user_prompt = user_prompt_template.format(input_text=user_input)

    response = mistral_llm.generate(system_prompt=formatted_system_prompt,
                                    user_prompt=formatted_user_prompt,
                                    max_tokens=max_tokens)
    return response

In [4]:
from pprint import pprint

user_input = "Посчитай количество калорий в 2 яйцахх"
response = generate_response(user_input)
pprint(response)

2024-11-23 14:00:55.997 | DEBUG    | __main__:_call:38 - Request Payload: {'model': 'mistral-large-latest', 'messages': [{'role': 'system', 'content': 'You are an assistant that helps with user requests.\nBe as helpful as possible and return the answer in Russian language.'}, {'role': 'user', 'content': 'Посчитай количество калорий в 2 яйцахх'}], 'max_tokens': 512}
2024-11-23 14:00:59.048 | DEBUG    | __main__:_call:41 - Response: ChatCompletion(id='f7d57ed730174e2aa0c395b490e86b59', choices=[Choice(finish_reason='error', index=0, logprobs=None, message=ChatCompletionMessage(content='Количество калорий в яйцах может варьировать в зависимости от их размера, но в среднем одно куриное яйцо среднего размера содержит около 70-80 калорий. Таким образом, два яйца будут содержать примерно 140-160 калорий.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1732359656, model='mistral-large-latest', object='chat.completion', service_tier=None, system_fin

('Количество калорий в яйцах может варьировать в зависимости от их размера, но '
 'в среднем одно куриное яйцо среднего размера содержит около 70-80 калорий. '
 'Таким образом, два яйца будут содержать примерно 140-160 калорий.')


In [ ]:
class MistralEmbed:
    def __init__(self, api_key: str, model_name: str, api_url: str):
        self.api_key = api_key
        self.model_name = model_name
        self.api_url = api_url

    @property
    def _model_type(self) -> str:
        return "mistral-embed"

    def _call(self, texts: List[str], **kwargs) -> List[List[float]]:
        client = openai.Client(api_key=self.api_key, base_url=self.api_url)
        payload = {
            "model": self.model_name,
            "input": texts,
            **kwargs
        }
        logger.debug("Request Payload: {}", payload)
        try:
            response = client.embeddings.create(**payload)
            logger.debug("Response: {}", response)
            return [embedding.embedding for embedding in response.data]
        except Exception as e:
            logger.error("Error: {}", e)
            raise

    def generate_embeddings(self, texts: Union[str, List[str]], **kwargs) -> Union[List[float], List[List[float]]]:
        if isinstance(texts, str):
            texts = [texts]
        return self._call(texts, **kwargs)

In [5]:
## testing functions

def split_text_into_chunks(text: str, chunk_size: int = 1024) -> List[str]:
    """
    Splits a large text into chunks of a specified size.

    Args:
        text (str): The input text to be split.
        chunk_size (int): The size of each chunk. Default is 1024.

    Returns:
        List[str]: A list of text chunks.
    """
    chunks = []
    for start in range(0, len(text), chunk_size):
        end = start + chunk_size
        chunks.append(text[start:end])
    return chunks

def read_file_and_split_into_chunks(file_path: str, chunk_size: int = 1024) -> List[str]:
    """
    Reads a file and splits its content into chunks of a specified size.

    Args:
        file_path (str): The path to the input file.
        chunk_size (int): The size of each chunk. Default is 1024.

    Returns:
        List[str]: A list of text chunks.
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return split_text_into_chunks(text, chunk_size)

In [6]:
def split_into_batches(chunks, batch_size):
    return [chunks[i:i + batch_size] for i in range(0, len(chunks), batch_size)]

In [ ]:
mistral_embed = MistralEmbed(api_key=MISTRAL_API_KEY,
                             model_name="mistral-embed",
                             api_url=MISTRAL_API_URL)


file_path = '../data/hmao_npa.txt'
chunks = read_file_and_split_into_chunks(file_path)

batch_size = 24

batches = split_into_batches(chunks, batch_size)

all_embeddings = []
for batch in batches:
    embeddings = mistral_embed.generate_embeddings(batch)
    all_embeddings.extend(embeddings)

for embedding in all_embeddings:
    print(len(embedding), embedding[:10])

In [ ]:
from langchain_community.document_loaders import WebBaseLoader


